# DATA PREPARATION

### Download Kaggle dataset 

In [1]:
import kagglehub
import shutil
import os

dataset_path = '../datasets/raw'
force_download = False

# Download latest version
path = kagglehub.dataset_download("soumendraprasad/musical-instruments-sound-dataset", force_download=force_download)
print("Kaggle downloaded files:", path)

# Cria a pasta de destino, se não existir
os.makedirs(dataset_path, exist_ok=True)

shutil.copytree(
    src=path,
    dst=dataset_path,
    dirs_exist_ok=True  # Permite copiar para pasta existente
)

print("Moved kaggle files to:", path)

/home/rcalabro/codebase/senac-ia/neural-networks/rn-musical-insturments/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Kaggle downloaded files: /home/rcalabro/.cache/kagglehub/datasets/soumendraprasad/musical-instruments-sound-dataset/versions/3
Moved kaggle files to: /home/rcalabro/.cache/kagglehub/datasets/soumendraprasad/musical-instruments-sound-dataset/versions/3


### Organize files

In [16]:
import os
import re
import shutil
import pandas as pd
from pathlib import Path

RAW_DATASET_PATH = '../datasets/raw'
NORMALIZED_DATASET_PATH = '../datasets/normalized'
AUGMENTED_DATASET_PATH = '../datasets/augmented'
PREPARED_DATASET_PATH = '../datasets/prepared'

os.makedirs(NORMALIZED_DATASET_PATH, exist_ok=True)

# Função para normalizar nomes dos arquivos
def normalize_filename(filename: str) -> str:
    stem = Path(filename).stem
    stem = re.sub(r"[^a-zA-Z0-9]", "_", stem).lower()
    stem = re.sub(r"_+", "_", stem).strip("_")
    return f"{stem}.wav"

In [3]:
# Caminhos RAW
RAW_TRAIN_CSV_PATH = Path(os.path.join(RAW_DATASET_PATH, 'Metadata_Train.csv'))
print(f"RAW_TRAIN_CSV_PATH: {RAW_TRAIN_CSV_PATH}")

RAW_TRAIN_AUDIOS_DIR = Path(os.path.join(RAW_DATASET_PATH, 'Train_submission', 'Train_submission'))
print(f"RAW_TRAIN_AUDIOS_DIR: {RAW_TRAIN_AUDIOS_DIR}")

RAW_TEST_CSV_PATH = Path(os.path.join(RAW_DATASET_PATH, 'Metadata_Test.csv'))
print(f"RAW_TEST_CSV_PATH: {RAW_TEST_CSV_PATH}")

RAW_TEST_AUDIOS_DIR = Path(os.path.join(RAW_DATASET_PATH, 'Test_submission', 'Test_submission'))
print(f"RAW_TEST_AUDIOS_DIR: {RAW_TEST_AUDIOS_DIR}")

RAW_TRAIN_CSV_PATH: ../datasets/raw/Metadata_Train.csv
RAW_TRAIN_AUDIOS_DIR: ../datasets/raw/Train_submission/Train_submission
RAW_TEST_CSV_PATH: ../datasets/raw/Metadata_Test.csv
RAW_TEST_AUDIOS_DIR: ../datasets/raw/Test_submission/Test_submission


In [4]:
# Caminhos NORMALIZED
NORMALIZED_TRAIN_CSV_PATH = Path(os.path.join(NORMALIZED_DATASET_PATH, 'train_metadata.csv'))
print(f"NORMALIZED_TRAIN_CSV_PATH: {NORMALIZED_TRAIN_CSV_PATH}")

NORMALIZED_TRAIN_AUDIOS_DIR = Path(os.path.join(NORMALIZED_DATASET_PATH, 'train_data'))
print(f"NORMALIZED_TRAIN_AUDIOS_DIR: {NORMALIZED_TRAIN_AUDIOS_DIR}")

NORMALIZED_TEST_CSV_PATH = Path(os.path.join(NORMALIZED_DATASET_PATH, 'test_metadata.csv'))
print(f"NORMALIZED_TEST_CSV_PATH: {NORMALIZED_TEST_CSV_PATH}")

NORMALIZED_TEST_AUDIOS_DIR = Path(os.path.join(NORMALIZED_DATASET_PATH, 'test_data'))
print(f"NORMALIZED_TEST_AUDIOS_DIR: {NORMALIZED_TEST_AUDIOS_DIR}")


NORMALIZED_TRAIN_CSV_PATH: ../datasets/normalized/train_metadata.csv
NORMALIZED_TRAIN_AUDIOS_DIR: ../datasets/normalized/train_data
NORMALIZED_TEST_CSV_PATH: ../datasets/normalized/test_metadata.csv
NORMALIZED_TEST_AUDIOS_DIR: ../datasets/normalized/test_data


### Normalization

#### Normalize test files

In [5]:
# Leitura do CSV
df = pd.read_csv(RAW_TEST_CSV_PATH)

# Correção e normalização da coluna de classe
df["Class"] = (
    df["Class"]
    .str.lower()
    .str.replace(r"^sound_", "", regex=True)
    .str.replace(r"(?i)^guiatr$", "guitar", regex=True)
)

# Guarda o nome original antes da normalização
df["OriginalFilename"] = df["FileName"]

# Normaliza o nome dos arquivos
df["FileName"] = df["FileName"].apply(normalize_filename)

df.head()

,FileName,Class,OriginalFilename
0,acoustic_guitar_logo_13084.wav,guitar,acoustic-guitar-logo-13084.wav
1,guitar_chords_70663.wav,guitar,guitar-chords-70663.wav
2,guitar_intro_110935.wav,guitar,guitar-intro-110935.wav
3,guitar_solo_27194.wav,guitar,guitar-solo-27194.wav
4,guitar_solo_5999.wav,guitar,guitar-solo-5999.wav


In [6]:
os.makedirs(NORMALIZED_DATASET_PATH, exist_ok=True)
NORMALIZED_TEST_AUDIOS_DIR.mkdir(parents=True, exist_ok=True)

In [7]:
missing_test_files = []

for _, row in df.iterrows():
    original_file = RAW_TEST_AUDIOS_DIR / row["OriginalFilename"]
    normalized_file = NORMALIZED_TEST_AUDIOS_DIR / row["FileName"]

    if original_file.exists():
        shutil.copy2(original_file, normalized_file)
    else:
        missing_test_files.append(str(original_file))

print(f"missing: {missing_test_files}")

missing: []


In [8]:
df = df.drop(columns=["OriginalFilename"])
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(r"[^a-z0-9]+", "_", regex=True)
    .str.strip("_")
)

df.to_csv(NORMALIZED_TEST_CSV_PATH, index=False)

In [9]:
print(f"✅ Arquivos normalizados e copiados para: {NORMALIZED_TEST_AUDIOS_DIR}")
print(f"📄 Novo CSV salvo em: {NORMALIZED_TEST_CSV_PATH}")

if missing_test_files:
    print(f"⚠️ Arquivos ausentes ({len(missing_test_files)}):")
    for f in missing_test_files[:5]:
        print(f" - {f}")
else:
    print("🎉 Todos os arquivos foram encontrados e copiados com sucesso!")

✅ Arquivos normalizados e copiados para: ../datasets/normalized/test_data
📄 Novo CSV salvo em: ../datasets/normalized/test_metadata.csv
🎉 Todos os arquivos foram encontrados e copiados com sucesso!


#### Normalize Train Files

In [10]:
# Leitura do CSV
df_train = pd.read_csv(RAW_TRAIN_CSV_PATH)

# Correção e normalização da coluna de classe
df_train["Class"] = (
    df_train["Class"]
    .str.lower()
    .str.replace(r"^sound_", "", regex=True)
    .str.replace(r"(?i)^guiatr$", "guitar", regex=True)
)

# Guarda o nome original antes da normalização
df_train["OriginalFilename"] = df_train["FileName"]

# Normaliza o nome dos arquivos
df_train["FileName"] = df_train["FileName"].apply(normalize_filename)

df_train.head()


,FileName,Class,OriginalFilename
0,1_e1_major_00.wav,guitar,1-E1-Major 00.wav
1,1_e1_major_01.wav,guitar,1-E1-Major 01.wav
2,1_e1_major_02.wav,guitar,1-E1-Major 02.wav
3,1_e1_major_03.wav,guitar,1-E1-Major 03.wav
4,1_e1_major_04.wav,guitar,1-E1-Major 04.wav


In [11]:
os.makedirs(NORMALIZED_DATASET_PATH, exist_ok=True)
NORMALIZED_TRAIN_AUDIOS_DIR.mkdir(parents=True, exist_ok=True)

In [12]:
missing_train_files = []

for _, row in df_train.iterrows():
    original_file = RAW_TRAIN_AUDIOS_DIR / row["OriginalFilename"]
    normalized_file = NORMALIZED_TRAIN_AUDIOS_DIR / row["FileName"]

    if original_file.exists():
        shutil.copy2(original_file, normalized_file)
    else:
        missing_train_files.append(str(original_file))

print(f"missing: {missing_train_files}")


missing: []


In [13]:
df_train = df_train.drop(columns=["OriginalFilename"])
df_train.columns = (
    df_train.columns
    .str.strip()
    .str.lower()
    .str.replace(r"[^a-z0-9]+", "_", regex=True)
    .str.strip("_")
)

df_train.to_csv(NORMALIZED_TRAIN_CSV_PATH, index=False)


In [14]:
print(f"✅ Arquivos normalizados e copiados para: {NORMALIZED_TRAIN_AUDIOS_DIR}")
print(f"📄 Novo CSV salvo em: {NORMALIZED_TRAIN_CSV_PATH}")

if missing_train_files:
    print(f"⚠️ Arquivos ausentes ({len(missing_train_files)}):")
    for f in missing_train_files[:5]:
        print(f" - {f}")
else:
    print("🎉 Todos os arquivos foram encontrados e copiados com sucesso!")

✅ Arquivos normalizados e copiados para: ../datasets/normalized/train_data
📄 Novo CSV salvo em: ../datasets/normalized/train_metadata.csv
🎉 Todos os arquivos foram encontrados e copiados com sucesso!


### Data Augmentation

In [45]:
import librosa
import soundfile as sf
import numpy as np
from tqdm import tqdm
import random

SAMPLE_RATE = 22050
MAX_AUGS_PER_ORIGINAL = 4

# Versão do dataset preparado
AUGMENTATION_VERSION = "v1"

# Diretórios base
AUGMENTED_DATASET_VERSION_PATH = Path(os.path.join(AUGMENTED_DATASET_PATH, AUGMENTATION_VERSION))
AUGMENTED_DATASET_VERSION_PATH.mkdir(parents=True, exist_ok=True)
print(f"📁 Versão {AUGMENTATION_VERSION}: {AUGMENTED_DATASET_VERSION_PATH}")

AUGMENTED_TRAIN_AUDIOS_DIR = Path(os.path.join(AUGMENTED_DATASET_VERSION_PATH, 'train_data'))
AUGMENTED_TRAIN_AUDIOS_DIR.mkdir(parents=True, exist_ok=True)
AUGMENTED_TRAIN_CSV_PATH = AUGMENTED_DATASET_VERSION_PATH / "train_metadata.csv"
print(f"📁 Cortes de treino serão salvos em: {AUGMENTED_TRAIN_AUDIOS_DIR}")
print(f"📁 Metadados de treino: {AUGMENTED_TRAIN_CSV_PATH}")

AUGMENTED_TEST_AUDIOS_DIR = Path(os.path.join(AUGMENTED_DATASET_VERSION_PATH, 'test_data'))
AUGMENTED_TEST_AUDIOS_DIR.mkdir(parents=True, exist_ok=True)
AUGMENTED_TEST_CSV_PATH = AUGMENTED_DATASET_VERSION_PATH / "test_metadata.csv"
print(f"📁 Cortes de treino serão salvos em: {AUGMENTED_TRAIN_AUDIOS_DIR}")
print(f"📁 Metadados de teste: {AUGMENTED_TRAIN_CSV_PATH}")


📁 Versão v1: ../datasets/augmented/v1
📁 Cortes de treino serão salvos em: ../datasets/augmented/v1/train_data
📁 Metadados de treino: ../datasets/augmented/v1/train_metadata.csv
📁 Cortes de treino serão salvos em: ../datasets/augmented/v1/train_data
📁 Metadados de teste: ../datasets/augmented/v1/train_metadata.csv


In [40]:
AUGMENTATION_COMBINATIONS = {
    "original": [],
    "stretch_102": [("stretch", 1.02)],
    "pitch_up_0_5": [("pitch", 0.5)],
    "pitch_down_0_5": [("pitch", -0.5)],
    "noise_0_003": [("noise", 0.003)],
    "gain_0_95": [("gain", 0.95)],
    "gain_1_05": [("gain", 1.05)],

    "stretch_pitch_up": [("stretch", 1.02), ("pitch", 0.5)],
    "pitch_noise": [("pitch", -0.5), ("noise", 0.002)],
    "stretch_noise": [("stretch", 0.98), ("noise", 0.002)],
    "pitch_gain": [("pitch", 0.5), ("gain", 1.05)],
    "stretch_gain": [("stretch", 1.02), ("gain", 1.05)],
}

def augment_audio(y, sr, steps):
    y_aug = np.copy(y)
    for aug_type, param in steps:
        if aug_type == "stretch":
            y_aug = librosa.effects.time_stretch(y_aug, rate=param)
        elif aug_type == "pitch":
            y_aug = librosa.effects.pitch_shift(y_aug, sr=sr, n_steps=param)
        elif aug_type == "noise":
            noise = np.random.normal(0, param, len(y_aug))
            y_aug += noise
        elif aug_type == "gain":
            y_aug *= param
    return y_aug

#### Augment test data

In [46]:
df_test = pd.read_csv(NORMALIZED_TEST_CSV_PATH)
df_test.columns = df_test.columns.str.strip().str.lower().str.replace(r"[^a-z0-9]+", "_", regex=True)

augmented_test_records = []
grouped_test = df_test.groupby("class")

for class_name, group in tqdm(df_test.groupby("class"), desc="Test Augmentation"):
    combo_names = list(AUGMENTATION_COMBINATIONS.keys())
    combo_names.remove("original")

    for i, row in group.reset_index(drop=True).iterrows():
        original_path = NORMALIZED_TEST_AUDIOS_DIR / row["filename"]
        if not original_path.exists():
            continue

        y, sr = librosa.load(original_path, sr=SAMPLE_RATE)

        original_filename = f"{original_path.stem}__original.wav"
        output_original_path = AUGMENTED_TEST_AUDIOS_DIR / original_filename
        sf.write(output_original_path, y, sr, format="WAV")
        augmented_test_records.append({
            "file_name": original_filename,
            "class": class_name,
            "augmentation": "original"
        })

        shuffled_combos = random.sample(combo_names, k=len(combo_names))
        for j in range(MAX_AUGS_PER_ORIGINAL):
            combo = shuffled_combos[j % len(shuffled_combos)]
            steps = AUGMENTATION_COMBINATIONS[combo]

            y_aug = augment_audio(y, sr, steps)
            filename_aug = f"{original_path.stem}__{combo}.wav"
            output_path = AUGMENTED_TEST_AUDIOS_DIR / filename_aug

            sf.write(output_path, y_aug, sr, format="WAV")
            augmented_test_records.append({
                "file_name": filename_aug,
                "class": class_name,
                "augmentation": combo
            })


Test Augmentation: 100%|██████████| 4/4 [00:47<00:00, 11.76s/it]


In [47]:
df_augmented_test = pd.DataFrame(augmented_test_records)
df_augmented_test.to_csv(AUGMENTED_TEST_CSV_PATH, index=False)

print(f"📄 CSV de metadados de teste salvo em: {AUGMENTED_TEST_CSV_PATH}")
print(f"🎧 Total de arquivos augmentados no teste: {len(df_augmented_test)}")


📄 CSV de metadados de teste salvo em: ../datasets/augmented/v1/test_metadata.csv
🎧 Total de arquivos augmentados no teste: 400


#### Augment train data

In [49]:

df_train = pd.read_csv(NORMALIZED_TRAIN_CSV_PATH)
grouped = df_train.groupby("class")

augmented_train_records = []


for class_name, group in tqdm(df_train.groupby("class"), desc="Train Augmentation"):
    combo_names = list(AUGMENTATION_COMBINATIONS.keys())
    combo_names.remove("original")

    for i, row in group.reset_index(drop=True).iterrows():
        original_path = NORMALIZED_TRAIN_AUDIOS_DIR / row["filename"]
        if not original_path.exists():
            continue

        y, sr = librosa.load(original_path, sr=SAMPLE_RATE)

        original_filename = f"{original_path.stem}__original.wav"
        output_original_path = AUGMENTED_TRAIN_AUDIOS_DIR / original_filename
        sf.write(output_original_path, y, sr, format="WAV")
        augmented_train_records.append({
            "file_name": original_filename,
            "class": class_name,
            "augmentation": "original"
        })

        shuffled_combos = random.sample(combo_names, k=len(combo_names))
        for j in range(MAX_AUGS_PER_ORIGINAL):
            combo = shuffled_combos[j % len(shuffled_combos)]
            steps = AUGMENTATION_COMBINATIONS[combo]

            y_aug = augment_audio(y, sr, steps)
            filename_aug = f"{original_path.stem}__{combo}.wav"
            output_path = AUGMENTED_TRAIN_AUDIOS_DIR / filename_aug

            sf.write(output_path, y_aug, sr, format="WAV")
            augmented_train_records.append({
                "file_name": filename_aug,
                "class": class_name,
                "augmentation": combo
            })

Train Augmentation: 100%|██████████| 4/4 [20:32<00:00, 308.17s/it]


In [51]:
df_augmented_train = pd.DataFrame(augmented_train_records)
df_augmented_train.to_csv(AUGMENTED_TRAIN_CSV_PATH, index=False)

print(f"📄 CSV de metadados salvo em: {AUGMENTED_TRAIN_CSV_PATH}")
print(f"🎧 Total de arquivos augmentados: {len(df_augmented_train)}")

📄 CSV de metadados salvo em: ../datasets/augmented/v1/train_metadata.csv
🎧 Total de arquivos augmentados: 13145


### Preparation

In [ ]:
import librosa
import soundfile as sf
import numpy as np
from tqdm import tqdm

WINDOW_SIZE = 3.0    # segundos
OVERLAP = 1.5        # segundos
SAMPLE_RATE = 22050

PREPARATION_VERSION = "v1"
AUGMENTATION_VERSION_SOURCE = "v1"

# Diretórios base
PREPARED_DATASET_VERSION_PATH = Path(os.path.join(PREPARED_DATASET_PATH, PREPARATION_VERSION))
PREPARED_DATASET_VERSION_PATH.mkdir(parents=True, exist_ok=True)
print(f"📁 Versão {PREPARATION_VERSION}: {PREPARED_DATASET_VERSION_PATH}")

PREPARED_TRAIN_AUDIOS_DIR = Path(os.path.join(PREPARED_DATASET_VERSION_PATH, 'train_data'))
PREPARED_TRAIN_AUDIOS_DIR.mkdir(parents=True, exist_ok=True)
print(f"📁 Cortes de treino serão salvos em: {PREPARED_TRAIN_AUDIOS_DIR}")

PREPARED_TEST_AUDIOS_DIR = Path(os.path.join(PREPARED_DATASET_VERSION_PATH, 'test_data'))
PREPARED_TEST_AUDIOS_DIR.mkdir(parents=True, exist_ok=True)
print(f"📁 Cortes de teste serão salvos em: {PREPARED_TRAIN_AUDIOS_DIR}")

AUGMENTED_PREPARE_DATASET_VERSION_PATH = Path(os.path.join(AUGMENTED_DATASET_PATH, AUGMENTATION_VERSION_SOURCE))


AUGMENTED_PREPARE_TEST_AUDIOS_DIR = Path(os.path.join(AUGMENTED_DATASET_VERSION_PATH, 'test_data'))
AUGMENTED_PREPARE_TEST_CSV_PATH = AUGMENTED_PREPARE_DATASET_VERSION_PATH / "test_metadata.csv"

AUGMENTED_PREPARE_TRAIN_AUDIOS_DIR = Path(os.path.join(AUGMENTED_DATASET_VERSION_PATH, 'train_data'))
AUGMENTED_PREPARE_TRAIN_CSV_PATH = AUGMENTED_PREPARE_DATASET_VERSION_PATH / "train_metadata.csv"


📁 Versão v1: ../datasets/prepared/v1
📁 Cortes de treino serão salvos em: ../datasets/prepared/v1/train_data
📁 Cortes de teste serão salvos em: ../datasets/prepared/v1/train_data


In [82]:
def slice_audio_with_overlap(file_path: Path, class_name: str, output_dir: Path, 
                             window_size: float, overlap: float, sample_rate: int = 22050):
    try:
        y, sr = librosa.load(file_path, sr=sample_rate)
    except Exception as e:
        print(f"❌ Erro ao carregar {file_path.name}: {e}")
        return []

    window_samples = int(window_size * sr)
    hop_samples = int((window_size - overlap) * sr)

    segments = []
    base_name = file_path.stem  # Nome base sem extensão

    for i, start in enumerate(range(0, len(y), hop_samples)):
        end = start + window_samples
        slice_audio = y[start:end]

        # Padding com zeros se for menor que o necessário
        if len(slice_audio) < window_samples:
            slice_audio = np.pad(slice_audio, (0, window_samples - len(slice_audio)))

        # Gera nome padronizado com extensão .wav
        filename_seq = f"{base_name}_{i+1:06}.wav"
        out_path = output_dir / filename_seq

        try:
            sf.write(out_path, slice_audio, sr, format='WAV')
            segments.append((filename_seq, class_name))
        except Exception as e:
            print(f"❌ Erro ao salvar {out_path.name}: {e}")

        if end >= len(y):
            break

    return segments


#### Prepare test files

In [83]:
df_test = pd.read_csv(NORMALIZED_TEST_CSV_PATH)

prepared_test_data = []

for _, row in tqdm(df_test.iterrows(), total=len(df_test)):
    original_path = NORMALIZED_TEST_AUDIOS_DIR / row["filename"]
    class_name = row["class"]

    if original_path.exists():
        segments = slice_audio_with_overlap(
            file_path=original_path,
            class_name=class_name,
            output_dir=PREPARED_TEST_AUDIOS_DIR,
            window_size=WINDOW_SIZE,
            overlap=OVERLAP,
            sample_rate=SAMPLE_RATE
        )
        prepared_test_data.extend(segments)


100%|██████████| 80/80 [00:06<00:00, 12.18it/s]


In [84]:
df_prepared_test = pd.DataFrame(prepared_test_data, columns=["filename", "class"])

PREPARED_TEST_CSV_PATH = PREPARED_DATASET_VERSION_PATH / "test_metadata.csv"
df_prepared_test.to_csv(PREPARED_TEST_CSV_PATH, index=False)

print(f"📄 Metadata de teste salva em: {PREPARED_TEST_CSV_PATH}")
print(f"🎧 Total de segmentos: {len(df_prepared_test)}")


📄 Metadata de teste salva em: ../datasets/prepared/v1/test_metadata.csv
🎧 Total de segmentos: 1238


#### Prepare train files

In [86]:
prepared_train_data = []

for _, row in tqdm(df_train.iterrows(), total=len(df_train)):
    original_path = NORMALIZED_TRAIN_AUDIOS_DIR / row["filename"]
    class_name = row["class"]

    if original_path.exists():
        segments = slice_audio_with_overlap(
            file_path=original_path,
            class_name=class_name,
            output_dir=PREPARED_TRAIN_AUDIOS_DIR,
            window_size=WINDOW_SIZE,
            overlap=OVERLAP,
            sample_rate=SAMPLE_RATE
        )
        prepared_train_data.extend(segments)


100%|██████████| 2629/2629 [02:51<00:00, 15.30it/s]


In [87]:
df_prepared_train = pd.DataFrame(prepared_train_data, columns=["file_name", "class"])

PREPARED_TRAIN_CSV_PATH = PREPARED_DATASET_VERSION_PATH / "train_prepared_metadata.csv"
df_prepared_train.to_csv(PREPARED_TRAIN_CSV_PATH, index=False)

print(f"📄 Metadata de treino salva em: {PREPARED_TRAIN_CSV_PATH}")
print(f"🎧 Total de segmentos: {len(df_prepared_train)}")


📄 Metadata de treino salva em: ../datasets/prepared/v1/train_prepared_metadata.csv
🎧 Total de segmentos: 29893
